<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

np.random.seed(1)

transform_list = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor()
            #transforms.Normalize([0.5,],[0.5,])
        ])
train = datasets.CIFAR10("./", train=True, transform=transform_list, download=True)

test = torch.utils.data.DataLoader(
    datasets.CIFAR10("./", train=False, transform=transform_list, download=True), batch_size = 32, shuffle = True
)

print("datasets loaded")

IMG_SIZE = 32*32

'''for data in train:
  X, y = data[0][0], data[1][0]
  plt.imshow(X.view(32, 32))
  print("Label: " + str(y))
  plt.show()
  break'''

class LogisticRegression(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegression, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)

  def forward(self, x):
    return self.linear(x)

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

model = LogisticRegression(IMG_SIZE, 10).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)



for epoch in range(2):
  train_size = int(0.9 * len(train))
  val_size = len(train) - train_size
  train_dataset, val_dataset = torch.utils.data.random_split(train, [train_size, val_size])
  train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 32, shuffle = True)
  for data in train_loader:
    X, y = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    output = model(X.view(-1, IMG_SIZE))
    loss = F.cross_entropy(output, y)
    loss.backward()
    optimizer.step()
  print(f'epoch: {epoch}, loss: {loss}')

def test_perform():
  total = 0
  correct = 0

  y_true = []
  y_pred = []
  with torch.no_grad():
    for data in test:
      X, y = data[0].to(device), data[1].to(device)

      output = model(X.view(-1, IMG_SIZE))
      _, predicted = torch.max(output, 1)

      total += y.size(0)
      correct += (predicted == y).sum().item()

      y_pred.extend(predicted.tolist())
      y_true.extend(y.tolist())

    accuracy = 100 * correct / total
    return accuracy, y_true, y_pred
    
accuracy, y_true, y_pred = test_perform()

print(confusion_matrix(y_true, y_pred))
print(f'Accuracy of the network on the 10000 test images: {accuracy}')

Files already downloaded and verified
Files already downloaded and verified
datasets loaded
Running on the GPU
epoch: 0, loss: 3.834385871887207
epoch: 1, loss: 2.5606110095977783
[[  0   2 853   3   1   9  12   0  16 104]
 [  0  49 495   4   7  23  22   0  27 373]
 [  0   2 876   5   4  45  24   0   2  42]
 [  0   6 724  26   5 122  23   0   7  87]
 [  0   2 865   6  10  50  12   0   2  53]
 [  0   1 691  11   1 224  14   0   8  50]
 [  0   3 767   8   8  69  55   0   5  85]
 [  0   0 801   2   4  56   8   0   9 120]
 [  0   5 697   3   0  30   7   0  65 193]
 [  0   7 381   3   4  17  16   0  27 545]]
Accuracy of the network on the 10000 test images: 18.5
